# Transaction Fraud Detection - Model Evaluation
## Comprehensive Model Comparison and Analysis

This notebook performs detailed evaluation and comparison of all trained models:
- Load trained models and results
- Confusion matrices and classification reports
- ROC curves and PR curves comparison
- Feature importance analysis
- Threshold tuning for optimal fraud detection
- Final model selection and recommendations

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
from sklearn.metrics import (
    confusion_matrix, classification_report, 
    roc_curve, auc, precision_recall_curve,
    average_precision_score
)

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Libraries imported successfully")

## 2. Load Trained Models and Results

In [ ]:
# Load all model results
with open('all_model_results.pkl', 'rb') as f:
    all_results = pickle.load(f)

# Load train/val split
with open('train_val_split.pkl', 'rb') as f:
    split_data = pickle.load(f)
    
X_val = split_data['X_val']
y_val = split_data['y_val']

print(f"✓ Loaded {len(all_results)} model results")
print(f"✓ Validation set: {X_val.shape[0]} samples")
print(f"✓ Fraud rate in validation: {y_val.mean():.2%}")

## 3. Performance Summary

In [ ]:
# Create summary DataFrame
summary_df = pd.DataFrame([{
    'Model': r['model_name'],
    'Accuracy': f"{r['accuracy']:.4f}",
    'Precision': f"{r['precision']:.4f}",
    'Recall': f"{r['recall']:.4f}",
    'F1-Score': f"{r['f1']:.4f}",
    'ROC-AUC': f"{r['roc_auc']:.4f}" if r['roc_auc'] else 'N/A',
    'Training Time': f"{r['training_time']:.2f}s"
} for r in all_results])

print(f"\n{'='*100}")
print("MODEL PERFORMANCE SUMMARY")
print(f"{'='*100}\n")
print(summary_df.to_string(index=False))

# Highlight best performers
print(f"\n{'='*100}")
print("🏆 BEST PERFORMERS")
print(f"{'='*100}")

metrics = ['f1', 'recall', 'precision', 'roc_auc']
metric_names = ['F1-Score', 'Recall (Fraud Detection)', 'Precision', 'ROC-AUC']

for metric, name in zip(metrics, metric_names):
    if metric == 'roc_auc':
        valid_results = [r for r in all_results if r[metric] is not None]
        if not valid_results:
            continue
        best = max(valid_results, key=lambda x: x[metric])
    else:
        best = max(all_results, key=lambda x: x[metric])
    print(f"Best {name:25s}: {best['model_name']:30s} ({best[metric]:.4f})")

## 4. Confusion Matrices

In [ ]:
# Plot confusion matrices for all models
n_models = len(all_results)
n_cols = 3
n_rows = (n_models + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
axes = axes.flatten() if n_models > 1 else [axes]

for idx, result in enumerate(all_results):
    cm = result['confusion_matrix']
    ax = axes[idx]
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=['Legitimate', 'Fraud'],
                yticklabels=['Legitimate', 'Fraud'])
    
    ax.set_title(f"{result['model_name']}\nF1: {result['f1']:.4f} | Recall: {result['recall']:.4f}", 
                 fontsize=12, fontweight='bold')
    ax.set_ylabel('True Label')
    ax.set_xlabel('Predicted Label')

# Hide extra subplots
for idx in range(n_models, len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('confusion_matrices_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Confusion matrices saved: confusion_matrices_comparison.png")

## 5. ROC Curves Comparison

In [ ]:
plt.figure(figsize=(12, 8))

for result in all_results:
    if result['roc_auc'] is not None:
        fpr, tpr, _ = roc_curve(y_val, result['y_pred_proba'])
        plt.plot(fpr, tpr, linewidth=2, 
                label=f"{result['model_name']} (AUC = {result['roc_auc']:.4f})")

plt.plot([0, 1], [0, 1], 'k--', linewidth=2, label='Random Classifier')
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate (Recall)', fontsize=12)
plt.title('ROC Curves - All Models', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('roc_curves_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ ROC curves saved: roc_curves_comparison.png")

## 6. Precision-Recall Curves

In [ ]:
plt.figure(figsize=(12, 8))

for result in all_results:
    if result['y_pred_proba'] is not None:
        precision, recall, _ = precision_recall_curve(y_val, result['y_pred_proba'])
        avg_precision = average_precision_score(y_val, result['y_pred_proba'])
        plt.plot(recall, precision, linewidth=2,
                label=f"{result['model_name']} (AP = {avg_precision:.4f})")

# Baseline (random classifier)
baseline = y_val.mean()
plt.axhline(y=baseline, color='k', linestyle='--', linewidth=2, 
           label=f'Random Classifier (AP = {baseline:.4f})')

plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title('Precision-Recall Curves - All Models', fontsize=14, fontweight='bold')
plt.legend(loc='best', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('precision_recall_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Precision-Recall curves saved: precision_recall_curves.png")

## 7. Feature Importance Analysis

In [ ]:
# Get feature names
feature_names = X_val.columns.tolist()

# Models with feature importance
importance_models = ['Decision Tree', 'Random Forest', 'XGBoost', 'LightGBM', 'Gradient Boosting']
importance_results = [r for r in all_results if r['model_name'] in importance_models]

if importance_results:
    n_importance_models = len(importance_results)
    fig, axes = plt.subplots(n_importance_models, 1, figsize=(14, 6*n_importance_models))
    
    if n_importance_models == 1:
        axes = [axes]
    
    for idx, result in enumerate(importance_results):
        model = result['model']
        
        # Get feature importance
        if hasattr(model, 'feature_importances_'):
            importances = model.feature_importances_
        else:
            continue
            
        # Create DataFrame and sort
        importance_df = pd.DataFrame({
            'Feature': feature_names,
            'Importance': importances
        }).sort_values('Importance', ascending=False).head(20)
        
        # Plot
        ax = axes[idx]
        ax.barh(range(len(importance_df)), importance_df['Importance'], color='steelblue')
        ax.set_yticks(range(len(importance_df)))
        ax.set_yticklabels(importance_df['Feature'])
        ax.invert_yaxis()
        ax.set_xlabel('Importance', fontsize=11)
        ax.set_title(f"Top 20 Features - {result['model_name']}", fontsize=13, fontweight='bold')
        ax.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('feature_importance_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✓ Feature importance plots saved: feature_importance_comparison.png")
else:
    print("⚠ No models with feature importance available")

## 8. Classification Reports

In [ ]:
print(f"\n{'='*100}")
print("DETAILED CLASSIFICATION REPORTS")
print(f"{'='*100}\n")

for result in all_results:
    print(f"\n{'─'*100}")
    print(f"{result['model_name']}")
    print(f"{'─'*100}")
    print(classification_report(y_val, result['y_pred'], 
                                target_names=['Legitimate (0)', 'Fraud (1)'],
                                digits=4))

## 9. Threshold Tuning for Best Model

In [ ]:
# Find best model by F1-score
best_model_result = max(all_results, key=lambda x: x['f1'])

print(f"\n{'='*100}")
print(f"THRESHOLD TUNING: {best_model_result['model_name']}")
print(f"{'='*100}\n")

if best_model_result['y_pred_proba'] is not None:
    # Test different thresholds
    thresholds = np.arange(0.1, 0.9, 0.05)
    metrics_by_threshold = []
    
    for threshold in thresholds:
        y_pred_thresh = (best_model_result['y_pred_proba'] >= threshold).astype(int)
        
        from sklearn.metrics import f1_score, precision_score, recall_score
        
        f1 = f1_score(y_val, y_pred_thresh)
        precision = precision_score(y_val, y_pred_thresh)
        recall = recall_score(y_val, y_pred_thresh)
        
        metrics_by_threshold.append({
            'Threshold': threshold,
            'F1-Score': f1,
            'Precision': precision,
            'Recall': recall
        })
    
    threshold_df = pd.DataFrame(metrics_by_threshold)
    
    # Plot
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Line plot
    ax = axes[0]
    ax.plot(threshold_df['Threshold'], threshold_df['F1-Score'], 
           marker='o', linewidth=2, label='F1-Score')
    ax.plot(threshold_df['Threshold'], threshold_df['Precision'], 
           marker='s', linewidth=2, label='Precision')
    ax.plot(threshold_df['Threshold'], threshold_df['Recall'], 
           marker='^', linewidth=2, label='Recall')
    ax.set_xlabel('Threshold', fontsize=12)
    ax.set_ylabel('Score', fontsize=12)
    ax.set_title('Metrics vs Threshold', fontsize=13, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Best thresholds
    best_f1_idx = threshold_df['F1-Score'].idxmax()
    best_recall_idx = threshold_df['Recall'].idxmax()
    
    ax = axes[1]
    bars = ['Default (0.5)', 'Best F1', 'Best Recall']
    
    default_metrics = threshold_df.iloc[(threshold_df['Threshold'] - 0.5).abs().argmin()]
    best_f1_metrics = threshold_df.iloc[best_f1_idx]
    best_recall_metrics = threshold_df.iloc[best_recall_idx]
    
    f1_scores = [default_metrics['F1-Score'], best_f1_metrics['F1-Score'], best_recall_metrics['F1-Score']]
    recalls = [default_metrics['Recall'], best_f1_metrics['Recall'], best_recall_metrics['Recall']]
    precisions = [default_metrics['Precision'], best_f1_metrics['Precision'], best_recall_metrics['Precision']]
    
    x = np.arange(len(bars))
    width = 0.25
    
    ax.bar(x - width, f1_scores, width, label='F1-Score', color='steelblue')
    ax.bar(x, recalls, width, label='Recall', color='coral')
    ax.bar(x + width, precisions, width, label='Precision', color='lightgreen')
    
    ax.set_ylabel('Score', fontsize=12)
    ax.set_title('Comparison of Threshold Strategies', fontsize=13, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(bars)
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('threshold_tuning_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Print results
    print(f"Default Threshold (0.5):")
    print(f"  F1-Score: {default_metrics['F1-Score']:.4f}")
    print(f"  Precision: {default_metrics['Precision']:.4f}")
    print(f"  Recall: {default_metrics['Recall']:.4f}")
    
    print(f"\nBest F1-Score Threshold ({best_f1_metrics['Threshold']:.2f}):")
    print(f"  F1-Score: {best_f1_metrics['F1-Score']:.4f}")
    print(f"  Precision: {best_f1_metrics['Precision']:.4f}")
    print(f"  Recall: {best_f1_metrics['Recall']:.4f}")
    
    print(f"\nBest Recall Threshold ({best_recall_metrics['Threshold']:.2f}):")
    print(f"  F1-Score: {best_recall_metrics['F1-Score']:.4f}")
    print(f"  Precision: {best_recall_metrics['Precision']:.4f}")
    print(f"  Recall: {best_recall_metrics['Recall']:.4f}")
    
    print("\n✓ Threshold tuning analysis saved: threshold_tuning_analysis.png")
else:
    print("⚠ Best model does not support probability predictions")

## 10. Model Comparison Visualization

In [ ]:
# Create comparison DataFrame
comparison_df = pd.DataFrame([{
    'Model': r['model_name'],
    'F1-Score': r['f1'],
    'Recall': r['recall'],
    'Precision': r['precision'],
    'Accuracy': r['accuracy']
} for r in all_results])

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics = ['F1-Score', 'Recall', 'Precision', 'Accuracy']
colors = ['steelblue', 'coral', 'lightgreen', 'mediumpurple']

for idx, (metric, color) in enumerate(zip(metrics, colors)):
    ax = axes[idx // 2, idx % 2]
    
    sorted_df = comparison_df.sort_values(metric, ascending=True)
    
    ax.barh(range(len(sorted_df)), sorted_df[metric], color=color, alpha=0.7)
    ax.set_yticks(range(len(sorted_df)))
    ax.set_yticklabels(sorted_df['Model'])
    ax.set_xlabel('Score', fontsize=11)
    ax.set_title(f'{metric} Comparison', fontsize=13, fontweight='bold')
    ax.grid(axis='x', alpha=0.3)
    
    # Add value labels
    for i, v in enumerate(sorted_df[metric]):
        ax.text(v + 0.005, i, f'{v:.4f}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig('model_comparison_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Model comparison visualization saved: model_comparison_metrics.png")

## 11. Final Recommendations

In [ ]:
print(f"\n{'='*100}")
print("FINAL MODEL SELECTION RECOMMENDATIONS")
print(f"{'='*100}\n")

# Best by different criteria
best_f1 = max(all_results, key=lambda x: x['f1'])
best_recall = max(all_results, key=lambda x: x['recall'])
best_precision = max(all_results, key=lambda x: x['precision'])

roc_results = [r for r in all_results if r['roc_auc'] is not None]
best_roc = max(roc_results, key=lambda x: x['roc_auc']) if roc_results else None

print("📊 Best Models by Metric:\n")
print(f"1. Best F1-Score (Balanced Performance):")
print(f"   Model: {best_f1['model_name']}")
print(f"   F1: {best_f1['f1']:.4f} | Precision: {best_f1['precision']:.4f} | Recall: {best_f1['recall']:.4f}\n")

print(f"2. Best Recall (Catch Most Frauds):")
print(f"   Model: {best_recall['model_name']}")
print(f"   Recall: {best_recall['recall']:.4f} | F1: {best_recall['f1']:.4f} | Precision: {best_recall['precision']:.4f}\n")

print(f"3. Best Precision (Minimize False Alarms):")
print(f"   Model: {best_precision['model_name']}")
print(f"   Precision: {best_precision['precision']:.4f} | F1: {best_precision['f1']:.4f} | Recall: {best_precision['recall']:.4f}\n")

if best_roc:
    print(f"4. Best ROC-AUC (Overall Discrimination):")
    print(f"   Model: {best_roc['model_name']}")
    print(f"   ROC-AUC: {best_roc['roc_auc']:.4f} | F1: {best_roc['f1']:.4f}\n")

print(f"\n{'─'*100}\n")
print("💡 Business Context Recommendations:\n")

print("For FRAUD DETECTION (High Stakes):")
print(f"  → Prioritize RECALL to catch maximum fraudulent transactions")
print(f"  → Recommended: {best_recall['model_name']} (Recall: {best_recall['recall']:.4f})")
print(f"  → Consider lowering threshold to increase recall further\n")

print("For BALANCED APPROACH:")
print(f"  → Use F1-Score to balance precision and recall")
print(f"  → Recommended: {best_f1['model_name']} (F1: {best_f1['f1']:.4f})")
print(f"  → Best overall performance for most use cases\n")

print("For MINIMAL FALSE ALARMS:")
print(f"  → Prioritize PRECISION when manual review is costly")
print(f"  → Recommended: {best_precision['model_name']} (Precision: {best_precision['precision']:.4f})")
print(f"  → Consider raising threshold to reduce false positives\n")

print(f"\n{'='*100}")
print("NEXT STEPS")
print(f"{'='*100}\n")

print("1. Load the best model based on your business requirements")
print("2. Apply threshold tuning if needed")
print("3. Make predictions on test_transaction.csv")
print("4. Monitor model performance in production")
print("5. Consider ensemble methods combining top models\n")

print(f"{'='*100}")
print("✅ EVALUATION COMPLETE - All visualizations saved!")
print(f"{'='*100}")